In [525]:
import random
import pandas as pd


In [526]:
numbers = [str(x) for x in range(10)]
CfarNumbers = set([])
def generate_CfarNr_LISA(): #Will generate an arbitrary unique 8 digit number
    while True:
        CfarNr_LISA = ""
        for _ in range(8):
            CfarNr_LISA = CfarNr_LISA + numbers[random.randint(0, 9)]
        if CfarNr_LISA not in CfarNumbers:
            CfarNumbers.add(CfarNr_LISA)
            return CfarNr_LISA

# for _ in range(100):
#     a = generate_CfarNr_LISA()
# print(CfarNumbers)

In [527]:
AstNumbers = set([])
def generate_AstNr_LISA(): #Gives a number to specify what job the person has, like a contractor, teacher, accountant etc
    not_allowed_number = ["00000"]
    generate_new_AstNr = random.randint(1,100)
    if generate_new_AstNr > 95: #5% of people have a number in these lines
        special_numbers = ["99980","99981","99982","99983","99984","99985","99986","99987","99988","99989","99990","99991","99992","99993","99994","99996","99998","99999"]
        return special_numbers[random.randint(0,len(special_numbers)-1)]
    else:
        if generate_new_AstNr > 50 and len(AstNumbers)>0:
            jobs = list(AstNumbers)
            return jobs[random.randint(0, len(jobs)-1)]
        else: 
            while True:
                AstNr_LISA = ""
                for _ in range(5):
                    AstNr_LISA = AstNr_LISA + numbers[random.randint(0, 9)]
                if AstNr_LISA not in AstNumbers and AstNr_LISA not in not_allowed_number:
                    AstNumbers.add(AstNr_LISA)
                    return AstNr_LISA

# for _ in range(100):
#     a = generate_AstNr_LISA()
# print(AstNumbers)    

In [528]:
PeOrgNumbers = set([])

def generate_PeOrgNr(personnummer):
    number = random.randint(0,10000)
    existing_job_numbers = list(PeOrgNumbers)
    if number > 87: #The person is a selfstarter and the organisation number is the person's personnumber
        PeOrgNumbers.add(personnummer[2:])
        return personnummer[2:]

    elif number > 60 and len(existing_job_numbers) > 0: # Use an existing job number 
        return existing_job_numbers[random.randint(0, len(existing_job_numbers)-1)]
    else: #Generate a new job
        while True:
            PeOrgNr = ""
            for _ in range(6):
                PeOrgNr = PeOrgNr + numbers[random.randint(0, 9)]
            PeOrgNr = PeOrgNr +"-"
            for _ in range(4):
                PeOrgNr = PeOrgNr + numbers[random.randint(0, 9)]
            if PeOrgNr not in PeOrgNumbers:
                PeOrgNumbers.add(PeOrgNr)
                return PeOrgNr
        

In [529]:
from variable_functions.generate_demographic_variables import generate_demographic
from variable_functions.generate_economic_variables import generate_economic
from variable_functions.generate_family_variables import generate_family
from variable_functions.generate_geographical_variables import generate_geographical
import variable_functions.generate_geographical_variables as ggv

In [530]:
def generate_company(personnummer, kommunnamn, lansnamn, prefix="", yrkstallning="", no_income=False):
    if no_income:
        work_data = {
                        f'{prefix}PeOrgNr'   : [None],
                        f'{prefix}Cfar_nr'   : [None],
                        f'{prefix}AstNr'     : [None],
                        f'{prefix}AstKommun' : [None],
                        f'{prefix}AstLan'    : [None],
                        f'{prefix}YrkStalln' : [None],
                        }     
        work_data = pd.DataFrame.from_dict(work_data)

    else:
        PeOrgNr = generate_PeOrgNr(personnummer)
        Cfar_Nr = generate_CfarNr_LISA()
        AstNr = generate_AstNr_LISA()
        work_data = {
                        f'{prefix}PeOrgNr'   : [PeOrgNr],
                        f'{prefix}Cfar_nr'   : [Cfar_Nr],
                        f'{prefix}AstNr'     : [AstNr],
                        f'{prefix}AstKommun' : [kommunnamn],
                        f'{prefix}AstLan'    : [lansnamn],
                        f'{prefix}YrkStalln' : [yrkstallning],
                        }     
        work_data = pd.DataFrame.from_dict(work_data)
    return work_data

In [531]:
def generate_working_ties(personnummer, county, economicstatus, yrkstallning):
    Kommun = ggv.generate_municipal(county)
    no_prefix_working_ties = generate_company(personnummer, Kommun, county, yrkstallning=str(yrkstallning))
    if economicstatus[0] > 0:
        Kommun = ggv.generate_municipal(county)
        prefix_working_ties1 = generate_company(personnummer, Kommun, county, prefix="KU1", yrkstallning="1") #Yrkstallning hardcoded needs FIX
    else:
        prefix_working_ties1 = generate_company(personnummer, Kommun, county, prefix="KU1", yrkstallning="1", no_income=True) #Yrkstallning hardcoded needs FIX

    if economicstatus[1] > 0:
        Kommun = ggv.generate_municipal(county)
        prefix_working_ties2 = generate_company(personnummer, Kommun, county, prefix="KU2", yrkstallning="1") #Yrkstallning hardcoded needs FIX
    else:
        prefix_working_ties2 = generate_company(personnummer, Kommun, county, prefix="KU2", yrkstallning="1", no_income=True) #Yrkstallning hardcoded needs FIX
    if economicstatus[2] > 0:
        Kommun = ggv.generate_municipal(county)
        prefix_working_ties3 = generate_company(personnummer, Kommun, county, prefix="KU3", yrkstallning="1") #Yrkstallning hardcoded needs FIX
    else:
        prefix_working_ties3 = generate_company(personnummer, Kommun, county, prefix="KU3", yrkstallning="1", no_income=True) #Yrkstallning hardcoded needs FIX
    
    working_data = pd.concat([no_prefix_working_ties, prefix_working_ties1, prefix_working_ties2, prefix_working_ties3], axis=1)
    return working_data


In [532]:
sample_year = 2019
amount = 1

In [533]:
Geographical = generate_geographical()
Lan = Geographical.loc[0, 'Lan']

Family = generate_family(sample_year, amount)
PersonNr = Family.loc[0, 'PersonNr']

Economic = generate_economic(amount, sample_year)
income = [Economic.loc[0, 'KU1lnk'], Economic.loc[0, 'KU2lnk'], Economic.loc[0, 'KU3lnk']]
yrks = Economic.loc[0, 'YrkStalln']

Demographic = generate_demographic(PersonNr)
Work = generate_working_ties(PersonNr, Lan, income, yrks)


1
